In [ ]:
# !wget https://raw.githubusercontent.com/mlzoo/multi-step-regression-tutorials/main/Raotbl6.csv
# !pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

df = pd.read_csv('Raotbl6.csv')
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df.drop('date', axis=1, inplace=True)

In [ ]:
# make future targets
for i in range(12):
    df['rgnp_{}'.format(i)] = df['rgnp'].shift(-i-1)

df.dropna(inplace=True)

In [ ]:
targets = [item for item in df.columns if 'rgnp_' in item]

X_train = df.drop(targets, axis=1)[: int(len(df) * 0.8)]
y_train = df[targets][: int(len(df) * 0.8)]

X_test = df.drop(targets, axis=1)[int(len(df) * 0.8) :]
y_test = df[targets][int(len(df) * 0.8) :]

model = MultiOutputRegressor(lgb.LGBMRegressor(objective='regression')).fit(X_train, y_train)

In [ ]:
pred = pd.DataFrame(model.predict(X_test), columns=targets)
pred.index = y_test.index
(y_test - pred).abs().mean()

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold 
import lightgbm as lgb

from sklearn.metrics import accuracy_score

# 训练数据集
train = pd.read_csv(...)
X_train = train[...]
y_train = train[...]

# 测试数据集
X_test = pd.read_csv(...)


# lightgbm参数
lgb_params = {
      "objective": "xxx", 
      "metric": "xxx", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 100,
      'learning_rate': 0.01,  
      'colsample_bytree':0.9, 
} 

# k-flod
kf = KFold(n_splits=5)

# 存储k-flod中的全部model
lgb_models = []

# 存储预测结果
y_pred = 0

for f,(tr_idx,val_idx) in enumerate(kf.split(X_tr_val)):
    
    # 分割训练集与验证集
    X_trn, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_trn, y_val = y_train.iloc[tr_idx], train.iloc[val_idx]
    
    # 生成lightgbm数据集
    lgb_train = lgb.Dataset(X_trn,y_trn)
    lgb_valid = lgb.Dataset(X_val,y_val)
    
    # 启动训练
    clf = lgb.train(params=lgb_params,  # 训练参数
                  train_set=lgb_train, # 训练集
                  valid_sets=[lgb_train, lgb_valid], # 衡量性能(训练集+验证集)
                  num_boost_round=10000, # 最大round
                  verbose_eval=100, # 每100round打印
                    )
    
    y_pred += model_binary.predict(X_test) / 5.0
    
    lgb_models.append(clf) 